In [1]:
#nbi:hide_in
import numpy as np
import ipywidgets as ipw
import matplotlib
import matplotlib.pyplot as plt
import sympy
from sympy import Symbol, nsolve, log, lambdify
import warnings
warnings.filterwarnings("ignore")


# Flory Huggins equation 

The Flory Huggins equation is given by:

$$
\Delta \bar F_{mix} =kT(\chi\phi(1.-\phi) + \phi/N_A\ln(\phi) + (1-\phi)/N_B*\ln(1-\phi))
$$

We will plot $\Delta \bar F_{mix}$ for various values of $N_A$,$N_B$, and $\chi$. For simplicity, we'll set $kT=1$ for the next section. 

## Common tangent construction

Now we would like to find the phase boundaries from the free energy expression given above. We can use the common tanget construction for that. Here, we solve it nummerically. Note that the spinodal and some special cases for the binodal have analytical solutions, as you have shown in previous homeworks.
The red dots correspond to the binodal ($\phi'$ and $\phi''$) and the green dots ($\phi_{sp}'$ and $\phi_{sp}''$) to the spinodal.


In [3]:
#nbi:hide_in
    
def F_mix(phi,NA,NB,chi,kT):
    return kT*(chi*phi*(1.-phi) + phi/NA*np.log(phi) + (1.-phi)/NB*np.log(1-phi))

def F_mix_s(phi,NA,NB,chi,kT):
    return kT*(chi*phi*(1.-phi) + phi/NA*log(phi) + (1.-phi)/NB*log(1-phi))

def d_F_mix_s(phi,NA,NB,chi,kT):
    return kT*(chi*(1.-2*phi) + 1/NA*log(phi)+ 1/NA -1/NB -1/NB*log(1-phi))

def solve_tangent(NA,NB,chi,kT):
    
    x1 = Symbol('x1', real=True)
    x2 = Symbol('x2', real=True)
    y1 = Symbol('y1', real=True)
    y2 = Symbol('y2', real=True)
    
    try:
        sol = nsolve((
        y1-F_mix_s(x1,NA,NB,chi,kT),
        y2-F_mix_s(x2,NA,NB,chi,kT), 
        d_F_mix_s(x1,NA,NB,chi,kT)-d_F_mix_s(x2,NA,NB,chi,kT),
        d_F_mix_s(x1,NA,NB,chi,kT)-(y2-y1)/(x2-x1)),
        (x1,x2,y1,y2),(0.01,0.99,-0.01,-0.01))
        
    except: 
        sol=[np.nan,np.nan,np.nan,np.nan]
    return sol 

def solve_spinodal(NA,NB,chi,kT):
    try:
        x1 = -(NA - NB - 2*NA*NB*chi + np.sqrt(-8*NA*NB**2*chi + (-NA + NB + 2*NA*NB*chi)**2))/(4*NA*NB*chi)
        x2 = (-NA + NB + 2*NA*NB*chi + np.sqrt(-8*NA*NB**2*chi + (-NA + NB + 2*NA*NB*chi)**2))/(4*NA*NB*chi)
        y1 = F_mix_s(x1,NA,NB,chi,kT)
        y2 = F_mix_s(x2,NA,NB,chi,kT)
        sol = [x1,x2,y1,y2]
    except:
        sol=[np.nan,np.nan,np.nan,np.nan]
    
    return sol 

def line(x,x1,x2,y1,y2):
    m = (y1-y2)/(x1-x2)
    return  m*(x-x1)+y1

def g3(NA=100,NB=100,chi=0.001):   
    kT = 1.0
    binodal = solve_tangent(NA,NB,chi,kT)
    spinodal = solve_spinodal(NA,NB,chi,kT)

    x = np.linspace(0.001,0.999,100)
   
    try:
        plt.plot(x,F_mix(x,NA,NB,chi,kT))
    except:
        pass 
    
    try:
        plt.scatter(binodal[0],binodal[2],c='red')
        plt.scatter(binodal[1],binodal[3],c='red')
    except:
        pass 
    
    try:
        plt.scatter(spinodal[0],spinodal[2],c='green')
        plt.scatter(spinodal[1],spinodal[3],c='green')
    except:
        pass
    
    try:
        plt.plot(x,line(x,*binodal),linestyle='--')
    except:
        pass 
    
    plt.xlabel('$\phi$')
    plt.ylabel('$\Delta F_{mix}$')
    
    plt.tight_layout()
    plt.show()

q = ipw.interactive(g3,NA=(10,1000,10),NB=(10,1000,10),chi=(0.00, 1, 0.0001));
display(q)

interactive(children=(IntSlider(value=100, description='NA', max=1000, min=10, step=10), IntSlider(value=100, …

In [4]:
def g4(NA=100,NB=100,chi=0.001):   
    kT = 1.0
    binodal = solve_tangent(NA,NB,chi,kT)
    spinodal = solve_spinodal(NA,NB,chi,kT)
    print(binodal[0],binodal[3])
    print(spinodal[0],spinodal[3])

q = ipw.interactive(g4,NA=(10,1000,10),NB=(10,1000,10),chi=(0.00, 1, 0.0001));
display(q)

interactive(children=(IntSlider(value=100, description='NA', max=1000, min=10, step=10), IntSlider(value=100, …